<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import psycopg2
import warnings

from projectutils.print_report import print_report

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
# Установление подключения
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

In [4]:
# Создаем пустой словарь для отчетов по результатам выполнения заданий
reports = {}

# Избавляемся от лишних предупреждений
warnings.simplefilter(action='ignore', category=UserWarning)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = '''
select
    count(v.id) vacancies_cnt
from vacancies v
'''
# результат запроса
vacancies_cnt = pd.read_sql_query(query_3_1, connection).iloc[0]['vacancies_cnt']
print(f'Количество вакансий в базе: {vacancies_cnt}')
reports['report_3_1'] = '''
В масштабах всего сервиса HeadHunter 49197 вакансий - это только частичная выборка.
Будем надеяться, что она репрезентативная, то есть в ней в равных пропорциях
присутствуют записи разных классов объектов 🙂. 
'''

Количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = '''
select
    count(e.id) employers_cnt
from employers e
'''
# результат запроса
employers_cnt = pd.read_sql_query(query_3_2, connection).iloc[0]['employers_cnt']
print(f'Количество работодателей в базе: {employers_cnt}')

reports['report_3_2'] = f'''
Учитывая, что общее количество юридических лиц и индивидуальных
предпринимателей в 2021 году достигло пордка 7 млн, и это только
в России, {employers_cnt} работодателей на этом фоне выглядят
более чем скромно. Это тоже частичная выборка. 🤔 Интересно будет
проверить её на пропуски и насколько она релевантна выборке
вакансий.
'''

Количество работодателей в базе: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [7]:
# текст запроса
query_3_3 = '''
select
    count(a.id) areas_cnt
from areas a
'''
# результат запроса
areas_cnt = pd.read_sql_query(query_3_3, connection).iloc[0]['areas_cnt']
print(f'Количество регионов в базе: {areas_cnt}')

reports['report_3_3'] = f'''
В России насчитывается 630 городских округов в то время как количество
городов (регионов) в базе составляет {areas_cnt}, из чего можно сделать
предположение, что в базе присутствуют не только регионы России, но и 
других стран 🌏.
'''

Количество регионов в базе: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [8]:
# текст запроса
query_3_4 = '''
select
    count(i.id) industries_cnt
from industries i
'''
# результат запроса
industries_cnt = pd.read_sql_query(query_3_4, connection).iloc[0]['industries_cnt']
print(f'Количество сфер деятельности в базе: {industries_cnt}')

reports['report_3_4'] = '''
Относительно количества сфер деятельности трудно сказать что либо определенное.
В открытых источниках не удалось отыскать количество таких сфер в каталоге HH 😢.
'''

Количество сфер деятельности в базе: 294


***

In [9]:
# выводы по предварительному анализу данных
print_report(reports, 'Выводы по предварительному анализу данных.', 3)

<h1>Раздел проекта: 3. Выводы по предварительному анализу данных.</h1><br>
    <div class='report'>
    <br><h4>
В масштабах всего сервиса HeadHunter 49197 вакансий - это только частичная выборка.
Будем надеяться, что она репрезентативная, то есть в ней в равных пропорциях
присутствуют записи разных классов объектов 🙂. 
</h4><br><h4>
Учитывая, что общее количество юридических лиц и индивидуальных
предпринимателей в 2021 году достигло пордка 7 млн, и это только
в России, 23501 работодателей на этом фоне выглядят
более чем скромно. Это тоже частичная выборка. 🤔 Интересно будет
проверить её на пропуски и насколько она релевантна выборке
вакансий.
</h4><br><h4>
В России насчитывается 630 городских округов в то время как количество
городов (регионов) в базе составляет 1362, из чего можно сделать
предположение, что в базе присутствуют не только регионы России, но и 
других стран 🌏.
</h4><br><h4>
Относительно количества сфер деятельности трудно сказать что либо определенное.
В открытых источниках не удалось отыскать количество таких сфер в каталоге HH 😢.
</h4></div>

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [16]:
# текст запроса
query_4_1 = '''
select
	a.name area,
	count(v.id) vac_cnt
from vacancies v
	join areas a on v.area_id=a.id
group by a.id
order by 2 desc
'''

# результат запроса
vac_cnt_by_area = pd.read_sql_query(query_4_1, connection)
display(vac_cnt_by_area.head(5))

# визуализация ТОП-5 регионов по количеству вакансий
fig = px.bar(
    vac_cnt_by_area.head(5),
    x='area', y='vac_cnt', color='area',
    title='Топ 5 регионов по количеству вакансий',
    labels={
		'area': 'Регион',
		'vac_cnt': 'Количество вакансий'	
	})
fig.update_layout(showlegend=False)
fig.write_image('./img/4_1.png')
fig.show()

# формируем отчет об инсайтах
msc_vac_cnt = vac_cnt_by_area.iloc[0]['vac_cnt']
spb_vac_cnt = vac_cnt_by_area.iloc[1]['vac_cnt']
ratio = round(msc_vac_cnt / spb_vac_cnt)
reports['report_4_1'] = f'''
<img src='./img/4_1.png'>

В разрезе распределения ваксансий по регионам следующие инсайты:
<ol>
<li>Москва, ожидаемо, на первом месте 👍🏻</li>
<li>В ТОП-5 регионов кроме столиц (включая историческую) России,
Беларуси и Казахстана вошла научная столица России - Новосибирск.
Поздравим ребят 🥳;</li>
<li>Количество вакансий в Санкт-Петебурге в {ratio} раза меньше,
чем в Москве. Питер - расти быстрее 💪🏻!</li>
<ol>
'''

,area,vac_cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [17]:
# текст запроса
query_4_2 = '''
select count(v.id) vac_cnt
from vacancies v
where v.salary_from is not NULL or v.salary_to is not NULL
'''

# результат запроса
vac_cnt = pd.read_sql_query(query_4_2, connection).iloc[0]['vac_cnt']
print(f'Количество вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой: {vac_cnt}')

ratio = vac_cnt / vacancies_cnt

reports['report_4_2'] = f'''
Соотношение количества вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой к
количству вакансий, в которых зарплата не указана вообще: {ratio:.2%}.
Получается что практически в половине случаев работодатели не указывают
в объявлении о вакансии ни нижнюю, ни верхнюю границы заработной платы 🙄.
Полагаю, что компании могут делать это с двумя целями:
<ol>
<li>Мотивировать соискателя на взаимодействие с HR-службой через вопрос: "А сколько буду получать 🤔?";</li>
<li>Не раскрывать своей политики в отношении формирования фонда оплаты труда и планов мотивации
конкурентам 🙊.</li>
<ol>
'''

Количество вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [18]:
# текст запроса
query_4_3 = '''
select 
	avg(v.salary_from) lower_salary_mean,
	avg(v.salary_to) upper_salary_mean
from vacancies v
'''

# результат запроса
salaries = pd.read_sql_query(query_4_3, connection)

lower_salary_mean = round(salaries.iloc[0]['lower_salary_mean'])
print(f'Среднее значение нижней границы зарплаты: {lower_salary_mean}')

upper_salary_mean = round(salaries.iloc[0]['upper_salary_mean'])
print(f'Среднее значение верхней границы зарплаты: {upper_salary_mean}')

ratio = upper_salary_mean / lower_salary_mean

reports['report_4_3'] = f'''
Верхняя граница заработной платы практически в {ratio:.2f} раз выше, чем нижняя 👈🏻.
Можно сделать вывод, что среднестатистически работодатели предполагают возможность
роста заработной платы сотрудника на {ratio-1:.0%} относительно начальной зарплаты
со временем. Но как известно, средние оценки - просто математический выкрутас,
который больше нацелен на сокращение количества информации, нежели на далеко идущие
выводы 🥸.
'''

Среднее значение нижней границы зарплаты: 71065
Среднее значение верхней границы зарплаты: 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
query_4_4 = '''
select 
	v.schedule||'—'||v.employment sched_empl_comp,
	count(v.id) vac_cnt
from vacancies v
group by v.schedule, v.employment
order by 2 desc
'''

# результат запроса
sched_empl_cnt = pd.read_sql_query(query_4_4, connection)
frst_schd_empl_cmp = sched_empl_cnt.iloc[0]['sched_empl_comp']
scnd_schd_empl_cmp = sched_empl_cnt.iloc[1]['sched_empl_comp']
print(f'На втором месте по популярности сочетания типа рабочего графика и типа трудоустройства стоит: {scnd_schd_empl_cmp}')

# визуализация ТОП-5 по популярности сочетаний типа рабочего графика и типа трудоустройства
fig = px.bar(
    sched_empl_cnt.head(5),
    x='sched_empl_comp', y='vac_cnt', color='sched_empl_comp',
    title='ТОП-5 по популярности сочетаний типа рабочего графика и типа трудоустройства',
    labels={
		'sched_empl_comp': 'График-занятость',
		'vac_cnt': 'Количество вакансий'	
	})
fig.update_layout(showlegend=False)
fig.write_image('./img/4_4.png')
fig.show()

# вывод отчета об интересном инсайте
frst_schd_empl_cnt = sched_empl_cnt.iloc[0]['vac_cnt']
scnd_schd_empl_cnt = sched_empl_cnt.iloc[1]['vac_cnt']
ratio = round(frst_schd_empl_cnt / scnd_schd_empl_cnt)
reports['report_4_4'] = f'''
<img src='./img/4_4.png'>

Вакансий самого популярного типа "Полный день — Полная занятость" в {ratio} раз больше,
чем вакансий идущим вторым по популярности типом "Удалённая работа — Полная занятость".
Вывод один: рынку удаленной работы еще расти и расти, а пока будем кататься на работу
утром и домой вечером, сжигая  огромные количества топлива 🔥, электроэнергии ⚡️ и теряя
драгоценное время ⏳ своей жизни в пробках. Но, эволюцию не удержать. Посмотрим на эти
показатели лет так через 10 🤔.
'''

На втором месте по популярности сочетания типа рабочего графика и типа трудоустройства стоит: Удаленная работа—Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [23]:
# текст запроса
query_4_5 = '''
select
	v.experience xp,
	count(v.id) vac_cnt
from vacancies v
group by v.experience
order by 2
'''

# результат запроса
experiences = pd.read_sql_query(query_4_5, connection)
display(experiences)

# визуализация распределения количества вакансий по требуемому опыту
fig = px.bar(
    experiences,
    x='xp', y='vac_cnt', color='vac_cnt',
    title='Распределение количества вакансий по требуемому опыту',
    labels={
		'xp': 'Треубемый опыт',
		'vac_cnt': 'Количество вакансий'	
	})
fig.update_layout(showlegend=False)
fig.update_traces(marker_coloraxis=None)
fig.write_image('./img/4_5.png')
fig.show()

# вывод отчета об интересном инсайте
six_plus_xp_cnt = experiences.iloc[0]['vac_cnt']
no_xp_cnt = experiences.iloc[1]['vac_cnt']
ratio = round(no_xp_cnt / six_plus_xp_cnt)
reports['report_4_5'] = f'''
<img src='./img/4_5.png'>

Парадоксально 🤔, что вакансий с требованием "Нет опыта" в {ratio} раз больше,
чем 	вакансий с требованием опыта более 6 лет. Можно сделать 3 вывода:
<ol>
<li>Компаний, которые готовы нанимать "дорогих" сотрудников с опытом более 6 лет
меньше на рынке. Скорее всего это крупный бизнес 🧐;</li>
<li>Компании предпочитают обучать людей специфике работы в своем бизнесе, нежели брать
людей с большим опытом работы в других компаниях. Для таким сотрудникам сложнее провести
онбординг, их тяжелее переобучить работать по-новому 🙄;</li>
<li>Руководители компаний не берут опытных сотрудников просто потому, что опасаются 😱
ослабления своих лидерских позиций, за счет проявления большей экспертности таких
новых сотрудников.</li>
</ol>
Так же примечательно, что самая популярная категория опыта работы в вакансиях 1-3 года.
'''

,xp,vac_cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [24]:
print_report(reports, 'Выводы по детальному анализу вакансий', 4)

<h1>Раздел проекта: 4. Выводы по детальному анализу вакансий</h1><br>
    <div class='report'>
    <br><h4>
<img src='./img/4_1.png'>

В разрезе распределения ваксансий по регионам следующие инсайты:
<ol>
<li>Москва, ожидаемо, на первом месте 👍🏻</li>
<li>В ТОП-5 регионов кроме столиц (включая историческую) России,
Беларуси и Казахстана вошла научная столица России - Новосибирск.
Поздравим ребят 🥳;</li>
<li>Количество вакансий в Санкт-Петебурге в 2 раза меньше,
чем в Москве. Питер - расти быстрее 💪🏻!</li>
<ol>
</h4><br><h4>
Соотношение количества вакансий, в которых заполнено хотя бы одно из двух полей с зарплатой к
количству вакансий, в которых зарплата не указана вообще: 48.93%.
Получается что практически в половине случаев работодатели не указывают
в объявлении о вакансии ни нижнюю, ни верхнюю границы заработной платы 🙄.
Полагаю, что компании могут делать это с двумя целями:
<ol>
<li>Мотивировать соискателя на взаимодействие с HR-службой через вопрос: "А сколько буду получать 🤔?";</li>
<li>Не раскрывать своей политики в отношении формирования фонда оплаты труда и планов мотивации
конкурентам 🙊.</li>
<ol>
</h4><br><h4>
Верхняя граница заработной платы практически в 1.56 раз выше, чем нижняя 👈🏻.
Можно сделать вывод, что среднестатистически работодатели предполагают возможность
роста заработной платы сотрудника на 56% относительно начальной зарплаты
со временем. Но как известно, средние оценки - просто математический выкрутас,
который больше нацелен на сокращение количества информации, нежели на далеко идущие
выводы 🥸.
</h4><br><h4>
<img src='./img/4_4.png'>

Вакансий самого популярного типа "Полный день — Полная занятость" в 5 раз больше,
чем вакансий идущим вторым по популярности типом "Удалённая работа — Полная занятость".
Вывод один: рынку удаленной работы еще расти и расти, а пока будем кататься на работу
утром и домой вечером, сжигая  огромные количества топлива 🔥, электроэнергии ⚡️ и теряя
драгоценное время ⏳ своей жизни в пробках. Но, эволюцию не удержать. Посмотрим на эти
показатели лет так через 10 🤔.
</h4><br><h4>
<img src='./img/4_5.png'>

Парадоксально 🤔, что вакансий с требованием "Нет опыта" в 5 раз больше,
чем 	вакансий с требованием опыта более 6 лет. Можно сделать 3 вывода:
<ol>
<li>Компаний, которые готовы нанимать "дорогих" сотрудников с опытом более 6 лет
меньше на рынке. Скорее всего это крупный бизнес 🧐;</li>
<li>Компании предпочитают обучать людей специфике работы в своем бизнесе, нежели брать
людей с большим опытом работы в других компаниях. Для таким сотрудникам сложнее провести
онбординг, их тяжелее переобучить работать по-новому 🙄;</li>
<li>Руководители компаний не берут опытных сотрудников просто потому, что опасаются 😱
ослабления своих лидерских позиций, за счет проявления большей экспертности таких
новых сотрудников.</li>
</ol>
Так же примечательно, что самая популярная категория опыта работы в вакансиях 1-3 года.
</h4></div>

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [25]:
# текст запроса
query_5_1 = '''
(select
	e.name employer,
	count(*) vac_cnt
from employers e
	join vacancies v on e.id = v.employer_id
group by e.id
order by 2 desc
limit 1)
union
(select
	e.name employer,
	count(*) vac_cnt
from employers e
	join vacancies v on e.id = v.employer_id
group by e.id
order by 2 desc
offset 4 limit 1)
order by 2 desc
'''
# результат запроса
empls = pd.read_sql_query(query_5_1, connection)
print('Это результат буквального выполнения задания')
display(empls)
print('Однако считаю результат малоинформативным. Добавляю немного отсебятины:')

Это результат буквального выполнения задания


,employer,vac_cnt
0,Яндекс,1933
1,Газпром нефть,331


Однако считаю результат малоинформативным. Добавляю немного отсебятины:


In [26]:
# модифицированный запрос
query_5_1_2 = '''
select
	e.name employer,
	count(*) vac_cnt
from employers e
	join vacancies v on e.id = v.employer_id
group by e.id
order by 2 desc
limit 5
'''

# Результат выполнения запроса
empls = pd.read_sql_query(query_5_1_2, connection)

# визуализация ТОП-5 работодателей по количеству вакансий
# визуализация распределения количества вакансий по требуемому опыту
fig = px.bar(
    empls,
    x='employer', y='vac_cnt', color='employer',
    title='ТОП-5 работодателей по количеству вакансий',
    labels={
		'employer': 'Работодатель',
		'vac_cnt': 'Количество вакансий'	
	})
fig.update_layout(showlegend=False)
fig.update_traces(marker_coloraxis=None)
fig.write_image('./img/5_1.png')
fig.show()

yandex_vac_cnt = empls.loc[0, 'vac_cnt']
rostelecom_vac_cnt = empls.loc[1, 'vac_cnt']
ratio = yandex_vac_cnt / rostelecom_vac_cnt
reports['report_5_1'] = f'''
<img src='./img/5_1.png'>

Яндекс уверенно лидирует 👍🏻 с количестом вакансий {yandex_vac_cnt}.
На втором месте идёт Ростелеком с количеством вакансий {rostelecom_vac_cnt}, что в {ratio: .2f} раз меньше,
чем у Яндекса. Далее следуют два работодателя из банковской сферы: Тинькофф и Сбер. Причём количество вакансий у Сбера меньше.
Пятерку лидеров замыкает Газпром нефть. Можно сделать 3 вывода:
<ol>
<li>На первом месте по количеству вакансиий идут ИТ-компании 💻;</li>
<li>В середине компании банковского сектора 🏦;</li>
<li>Последнее место в ТОП-5 у компании из нефтегазового сектора 🛢.</li>
</ol>
'''

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [27]:
# текст запроса
query_5_2 = '''
SELECT empl.area area, empl.empl_cnt empl_cnt, vac.vac_cnt vac_cnt FROM
	(SELECT
		ar.name area,
		COUNT (DISTINCT e.id) empl_cnt
	FROM areas ar
		LEFT JOIN employers e ON ar.id = e.area
	GROUP BY ar.id) as empl
	JOIN
	(SELECT
		ar.name area,
		COUNT (DISTINCT v.id) vac_cnt
	FROM areas ar
		LEFT JOIN vacancies v ON ar.id = v.area_id
	GROUP BY ar.id) as vac
ON empl.area = vac.area
WHERE vac_cnt = 0
ORDER BY empl_cnt DESC
LIMIT 1
'''
# результат запроса
empl_vac = pd.read_sql_query(query_5_2, connection)
display(empl_vac)
answer = empl_vac.loc[0, 'area']
reports['report_5_2'] = f'''
Регион без вакансий с максимальным количеством работодателей: {answer}
Вывод: в этом регионе сосредоточено большинство работодателей, вакансии которых присутствуют в базе данных,
но в самих вакансиях регионы указываются более точно 😑.
'''

,area,empl_cnt,vac_cnt
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [28]:
# текст запроса
query_5_3 = '''
SELECT
	e.name empl,
	COUNT(DISTINCT v.area_id) ar_cnt
FROM
	employers e
JOIN
	vacancies v ON e.id = v.employer_id
GROUP BY e.id
ORDER BY ar_cnt DESC
'''
# результат запроса
empl_area = pd.read_sql_query(query_5_3, connection)
display(empl_area.head())
answer = empl_area.loc[0, 'ar_cnt']
print(f'Максимальное значение количества регионов, в которых публикуются для одного : {answer}')

,empl,ar_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


Максимальное значение количества регионов, в которых публикуются для одного : 181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса
query_5_4 = '''
SELECT COUNT(*)
	FROM (SELECT
		e.name empl,
		COUNT(DISTINCT i.industry_id) ind_cnt
	FROM
		employers e
	LEFT JOIN
		employers_industries i ON e.id = i.employer_id
	GROUP BY e.id
	HAVING COUNT(DISTINCT i.industry_id) = 0) AS empl_ind
'''
# результат запроса
empl_ind = pd.read_sql_query(query_5_4, connection)
display(empl_ind.head())
answer = empl_ind.loc[0, 'count']
print(f'Количетво работодателей, у которых не указана сфера деятельности: {answer}')

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса
query_5_5 = '''
SELECT
	e.name empl,
	COUNT(DISTINCT i.industry_id) ind_cnt
FROM
	employers e
LEFT JOIN
	employers_industries i ON e.id = i.employer_id
GROUP BY e.id
HAVING COUNT(DISTINCT i.industry_id) = 4
ORDER BY empl
OFFSET 2
LIMIT 1
'''
# результат запроса
empl_ind = pd.read_sql_query(query_5_5, connection)
display(empl_ind.head())
answer = empl_ind.loc[0, 'empl']
print(f'Компания, находящаяся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности: {answer}')

Малоинформативно. Посмотрим, а что это за виды деятельсности.

In [ ]:
# текст запроса
query_5_5_1 = """
select i.name industry
from industries i
join employers_industries ei on i.id = ei.industry_id
join employers e on ei.employer_id = e.id
where e.name='2ГИС'
"""
# результат запроса
industries = pd.read_sql_query(query_5_5_1, connection)
display(industries.head())

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.

In [ ]:
# текст запроса
query_5_6 = '''
SELECT COUNT(DISTINCT empl_ind.empl)
FROM
	(SELECT
		ei.employer_id empl,
		i.name ind
	FROM
		employers_industries ei
	JOIN
		industries i ON ei.industry_id = i.id
	WHERE i.name='Разработка программного обеспечения') as empl_ind
'''
# результат запроса
empl_ind = pd.read_sql_query(query_5_6, connection)
display(empl_ind.head())
answer = empl_ind.loc[0, 'count']
print(f'Количество работодателей, у которых указана сфера деятельности "Разработка программного обеспечения": {answer}')

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников
from bs4 import BeautifulSoup
import requests

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
page = BeautifulSoup(response.text, 'html.parser')
cities = [a.get('title') for a in page.find_all('tbody')[1].find_all('a')[4:] if a.get('title') is not None]
val_cities = 'VALUES\n'
for city in cities:
    val_cities += f"('{city}'),\n"
val_cities = val_cities[:-2]
print(val_cities)

In [ ]:
# текст запроса
query_5_7 = '''
with city_vac_cnt as (
select a.name, COUNT(v.id) cnt
from vacancies v
join areas a on a.id = v.area_id
where v.employer_id in
	(select e.id
	from employers e
	where e.name = 'Яндекс')
	and
	v.area_id in
	(SELECT a.id
	 FROM areas a
	 where a.name in (VALUES
	 ('Москва'),
	 ('Санкт-Петербург'),
	 ('Новосибирск'),
	 ('Екатеринбург'),
	 ('Казань'),
	 ('Нижний Новгород'),
	 ('Челябинск'),
	 ('Красноярск'),
	 ('Самара'),
	 ('Уфа'),
	 ('Ростов-на-Дону'),
	 ('Омск'),
	 ('Краснодар'),
	 ('Воронеж'),
	 ('Пермь'),
	 ('Волгоград')))
group by a.name)

(select * from city_vac_cnt)
union
(select 'Total', sum(city_vac_cnt.cnt) from city_vac_cnt)
order by 2
'''
# результат запроса
result = pd.read_sql_query(query_5_7, connection)
display(result)
answer = result.shape[0]
print(f'Результирующая таблица содержит {answer} строк')
answer = int(result.tail(1)['cnt'])
print(f'Общее количество вакансий Яндекса в городах миллионниках: {answer}')

### Выводы по анализу работодателей

1. Лидером по количеству вакансий является Яндекс 👍🏻. Пятерку лидеров замыкает Газпром нефть. При этом по количеству вакансий Яндекс опережает Ростелеком, идущий на втором месте, почти в четыре 👀 раза;
2. Вакансий с регионом Россия нет ❌, при этом количество работодателей в нем рекордные 410 🤔;
3. По количеству регионов, в которых представлены ваксансии компаний первые два места также занимают Яндекс и Ростелеком соответственно 👌🏻;
4. Почти восемь с половиной тысяч работодателей не указали сферу деятельности 🙈. Видимо это HR-агентства, которые через размещение популярных вакансий собирают таким образом базу спецалистов, чтобы предлагать на её основе различные услуги своим корпоративным клиентам;
5. Компания 2ГИС указала 4 вида деятельности: производство контента, коналтинг, колл-центры, интернет-компания 🤔;
6. Более трех с половиной тысяч компаний указали в сфере деятельности разработку программного обеспечения . Видим, что борьба за специалистов в сфере ИТ идёт не шуточная 🤼‍♂️;
7. Из почти двух тысяч вакансий Яндекс только 485 размещены в городах миллионниках. Видимо организация работы на удаленке позволяет Яндекс охватить все регионы и повысить таким образом свои возможности в найме специалистов 👍🏻.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
query_6_1 = '''
select count(v.id) vac_cnt
from vacancies v
where lower(v.name) like '%data%' or lower(v.name) like '%данн%'
'''
# результат запроса
result = pd.read_sql_query(query_6_1, connection)
display(result)
ds_vac_cnt = result.iloc[0].vac_cnt
print(f'Вакансий, имеющих отношение к данным: {ds_vac_cnt}')
ratio = ds_vac_cnt / vacancies_cnt
print(f'Доля таких вакансий: {ratio: .2%}')

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса
query_6_2 = '''
select count(v.id) vac_cnt
from vacancies v
where
	(lower(v.name) like '%data scientist%' or
	lower(v.name) like '%data science%'  or
	lower(v.name) like '%исследователь данных%' or
	(v.name like '%ML%' and v.name not like '%HTML%') or
	lower(v.name) like '%machine learning%' or
	lower(v.name) like '%машинн%обучен%') and
	(lower(v.name) like '%junior%' or
	 v.experience = 'Нет опыта' or
	 v.employment = 'Стажировка')
'''
# результат запроса
result = pd.read_sql_query(query_6_2, connection)
display(result)
juniors_vac_cnt = result.iloc[0].vac_cnt
print(f'Вакансий, подходящих для начинающего дата-сайентиста: {juniors_vac_cnt}')
ratio = juniors_vac_cnt / ds_vac_cnt
print(f'Доля таких вакансий: {ratio:.2%}')

По условиям задания ответ правильный. Однако, давайте посмотрим, а только ли HTML не имеет отношения к ML.

In [ ]:
# текст запроса
query_6_2_1 = '''
select distinct v.name vacancy
from vacancies v
where 
	v.name like '%ML%' and v.name not like '%HTML%' and not
	(lower(v.name) like '%data scientist%' or
	lower(v.name) like '%data science%'  or
	lower(v.name) like '%исследователь данных%' or
	lower(v.name) like '%machine learning%' or
	lower(v.name) like '%машинн%обучен%')
'''
# результат запроса
result = pd.read_sql_query(query_6_2_1, connection)
pd.set_option('display.max_rows', 100)
display(result)

Видим, что при такой фильтрации мы принимаем вакансии, содержащие в названии QML (пример в позиции 1) и AML (пример в позиции 49). Посмотрим, каково действительное количество вакансий DS для начинающих специалистов: 

In [ ]:
# текст запроса
query_6_2_2 = '''
select count(v.id) vac_cnt
from vacancies v
where
	(lower(v.name) like '%data scientist%' or
	lower(v.name) like '%data science%'  or
	lower(v.name) like '%исследователь данных%' or
	(v.name like '%ML%' and not (v.name like '%HTML%' or v.name like '%QML%' or v.name like '%AML%')) or
	lower(v.name) like '%machine learning%' or
	lower(v.name) like '%машинн%обучен%') and
	(lower(v.name) like '%junior%' or
	 v.experience = 'Нет опыта' or
	 v.employment = 'Стажировка')
'''
# результат запроса
result = pd.read_sql_query(query_6_2_2, connection)
display(result)
answer = result.iloc[0].vac_cnt
print(f'Вакансий, подходящих для начинающего дата-сайентиста: {answer}')

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_6_3 = """
select count(v.id) vac_cnt
from vacancies v
where
	(lower(v.name) like '%data scientist%' or
	lower(v.name) like '%data science%'  or
	lower(v.name) like '%исследователь данных%' or
	(v.name like '%ML%' and v.name not like '%HTML%') or
	lower(v.name) like '%machine learning%' or
	lower(v.name) like '%машинн%обучен%') and
	(v.key_skills like '%SQL%' or
	 lower(v.key_skills) like '%postgres%')
"""
# результат запроса
result = pd.read_sql_query(query_6_3, connection)
display(result)
sql_vac_cnt = result.iloc[0].vac_cnt
print(f'Вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres: {sql_vac_cnt}')
ratio = sql_vac_cnt / ds_vac_cnt
print(f'Доля таких вакансий: {ratio:.2%}')


По условиям задания ответ 201 верен, однако проверим, а попали ли сюда вакансии с QML или AML:

In [ ]:
# текст запроса
query_6_3_1 = """
select count(v.id) vac_cnt
from vacancies v
where
	(lower(v.name) like '%data scientist%' or
	lower(v.name) like '%data science%'  or
	lower(v.name) like '%исследователь данных%' or
	(v.name like '%ML%' and not (v.name like '%HTML%' or v.name like '%QML%' or v.name like '%AML%')) or
	lower(v.name) like '%machine learning%' or
	lower(v.name) like '%машинн%обучен%') and
	(v.key_skills like '%SQL%' or
	 lower(v.key_skills) like '%postgres%')
"""
# результат запроса
result = pd.read_sql_query(query_6_3_1, connection)
display(result)
answer = result.iloc[0].vac_cnt
print(f'Вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres: {answer}')

Как видим попали в количестве 3 штук.

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = """
select count(v.id) vac_cnt
from vacancies v
where
	(lower(v.name) like '%data scientist%' or
	lower(v.name) like '%data science%'  or
	lower(v.name) like '%исследователь данных%' or
	(v.name like '%ML%' and not v.name like '%HTML%') or
	lower(v.name) like '%machine learning%' or
	lower(v.name) like '%машинн%обучен%') and
	(lower(v.key_skills) like '%python%')
"""
# результат запроса
result = pd.read_sql_query(query_6_4, connection)
display(result)
python_cnt = result.iloc[0].vac_cnt
print(f'Вакансий для DS, в которых в качестве ключевого навыка указан python: {python_cnt}')
ratio = python_cnt / ds_vac_cnt
print(f'Доля таких вакансий: {ratio:.2%}')

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса
query_6_5 = """
with skills as (
select (char_length(v.key_skills) - char_length(replace(v.key_skills, chr(9), '')) + 1) skills_cnt
from vacancies v
where
	lower(v.name) like '%data scientist%' or
	lower(v.name) like '%data science%'  or
	lower(v.name) like '%исследователь данных%' or
	(v.name like '%ML%' and not v.name like '%HTML%') or
	lower(v.name) like '%machine learning%' or
	lower(v.name) like '%машинн%обучен%')

select avg(s.skills_cnt) from skills s
"""
# результат запроса
result = pd.read_sql_query(query_6_5, connection)
display(result)
answer = result.iloc[0].avg
print(f'Вакансий для DS, в которых в качестве ключевого навыка указан python: {answer:.2f}')

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса
query_6_6 = """
with ds_vacs as (
	select v.experience, v.salary_from, v.salary_to
	from vacancies v
	where (v.salary_from is not null or v.salary_to is not null) and
	  (lower(v.name) like '%data scientist%' or
	   lower(v.name) like '%data science%'  or
	   lower(v.name) like '%исследователь данных%' or
	   (v.name like '%ML%' and not v.name like '%HTML%') or
	   lower(v.name) like '%machine learning%' or
	   lower(v.name) like '%машинн%обучен%'))

select v.experience, AVG(COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)) avg_salary
from ds_vacs v
group by v.experience
"""
# результат запроса
result = pd.read_sql_query(query_6_6, connection)
result.set_index('experience', inplace=True)
display(result)
answer = round(result.loc['От 3 до 6 лет', 'avg_salary'], 0)
print(f'Средняя заработная плата в Data Science с опытом работы от 3 до 6 лет: {answer}')

***

### Выводы по предметному анализу

1. Почти 1800 вакансий в сфере Data Science, что составляет 3,6% от общего количества, что не мало. Вывод: Data Science становится всё более популярным у работодателей;
2. При этом всего 51 вакансия подходящая для начинающих дата сайентистов, что составляет менее 3%. Вывод: всем компаниям нужны опытные специалисты. Мало кто готов возиться с новобранцами, обучать их, разбирать их ошибки постоянно отвлекаясь от работы;
2. В чуть более чем 11% вакансий требуется наличие навыков SQL или postgres. По-видимому наличие таких навыков для работодателей не является критически важным;
3. При этом требование знания Python встречается уже в почти 20%, то есть вдвое чаще. При этом учитывались только те вакансии, в которых требование наличия навыка Python был указан явно. Скрыто требование знания Python может быть отражено в требовании знания тех или иных библиотек, таких как Pandas, sklearn, matplotlib, seaborn и т.п.;
4. Среднее количество требуемых навыков в сфере Data Science чуть более 6. Да, в этой професси требуется много знать и постоянно учиться;
5. При этом средняя заработная плата для специалистов с опытом работы от 3 до 6 лет свыше 240 тысяч рублей является хорошим стимулом к профессиональному росту.

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

1. Выборка в базе данных ограниченная. Конечно в базе HeadHunter более чем 49 тысяч вакансий. Все выводы этого исследования будут близки к реальности только в случае, если эта выборка репрезентативна. То есть в ней в равной мере представлены различные классы объектов;
2. Количество работодателей так же указывает на то, что мы работаем с частичной выборкой данных;
3. В базе присутствуют вакансии и работодатели не только из России;
4. В разрезе вакансий по регионам ожидаемо первые три места занимают Москва, Санкт-Петербург и Минск. Пятерку замыкает Алматы. А четвертое место у Новосибирска. Главный наукоград России интенсивно развивается. Это радует;
5.  